In [1]:
# Import the functions 
import sys
sys.path.append('./00_Functions')

## Import Statements 
import geemap
import ee
import ipywidgets
import ipyleaflet

# Subfunctions
from f_Dates import dates
from f00_vizParamStore import vizParamStore
from f01_SAR import SAR
from f02_FloodExtent import FloodExtent
from f03_Population import Population
from f04_Agriculture import Agriculture
from f05_Urban import Urban

In [2]:
# Only Run Once - Authenticates your Google Earth Engine Instance
#ee.Authenticate()
# Initiates the instance
ee.Initialize()

# Synthetic Aperture Radar Flood Mapping

Synthetic Aperture Radar (SAR) is an actively sensed (it uses energy to generate the signal, in this case radar) data collection method. This application will use a change detection approach to generate a flood extent map using Sentinel-1 'Ground Range Detected' imagery. 

This consists of generating a before and after flood event elevation model, which we can use raster mathematics to obtain a difference map based on the set threshold. Ground Range Detected imagery includes the following preprocessing steps: 

* Thermal-Noise Removal
* Radiometric calibration
* Terrain-correction

For this app, the user will specify the pass difference, polarization and difference threshold. 

The sample script builds a map of flooding in Beira, Mozambique where a cyclone caused a significant coastal flooding in March 2019. More than 200k people were affected. 

## User Inputs

For this application, the user will have to input several components

### Geometry
You can build a polygon in the Google Earth Engine Code editor and input the code directly into the script. 

### Time Period
* Set start and end dates of a period BEFORE the flood
* Make sure it is long enough for Sentinel-1 to acquire an image (repitition rate = 6 days)
* Adjust these parameters, if ImageCollection does not contain any elements

### SAR Parameters

##### `polarization`
* VH mostly is the prefered polarization for flood mapping
* However, it always depends on your study area, you can select 'VV' as well

##### `pass_direction` 
* Choose either "DESCENDING" or ASCENDING' 
* When images are being compared use only one pass direction
* Change this parameter if your image collection is empty
* In some areas more Ascending images exist than than descending and vice-versa
                           
##### `difference_threshold` = 1.25
* Threshold to calculate difference between before/after flood image
* Built with trial and error
* Adjust if you have significant false positives or negatives 
                           
##### `relative_orbit` = 79
* Empty by default 
* Used if you know the relative orbit for your study area, 
* It will  filter the image collection to avoid errors caused by comparing different relative orbits

In [3]:
### Flooding in Beira, Mozambique ###

geometry = ee.Geometry.Polygon(
    [[
        [35.53377589953368, -19.6674648789114],
        [34.50106105578368, -18.952058786515526],
        [33.63314113390868, -19.87423907259203],
        [34.74825343859618, -20.61123742951084]
    ]])

before_start= '2019-03-01'
before_end='2019-03-10'

after_start='2019-03-10'
after_end='2019-03-23'

polarization = "VH" 
pass_direction = "DESCENDING"
difference_threshold = 1.25
#relative_orbit = 79;

center = [-19.7234, 34.598]
zoom = 10

In [4]:
aoi = ee.FeatureCollection(geometry);

In [5]:
before_filtered, after_filtered = SAR(geometry, polarization, pass_direction, difference_threshold,
           before_start, before_end, after_start, after_end)

## Calculations
The primary calculations are handled in Python subfunctions. More detail can be found in each of the individual scripts, but there are five major segments.

#### Difference 
Calculates the difference betwen the SAR images.
#### Flood Extent
Calculates the extent of the flooding via the difference between the SAR images and the difference threshold used.
#### Population 
Uses the JRC Global Human Settlement Popluation Density layer to identify the population affected. 
#### Agriculture 
Uses MODIS Land Cover Type Yearly Global 500m to identify the agricultural areas affected. 
#### Urban Areas
Uses MODIS Land Cover Type Yearly Global 500m to identify the urban areas affected. 

In [6]:
### Chained Subfunctions
difference, flooded, flood_area_ha = FloodExtent(before_filtered, after_filtered, difference_threshold, aoi, polarization)
population_count, population_exposed, number_pp_exposed = Population(aoi, flooded)
cropland, cropland_affected, crop_area_ha, LC, flooded_res = Agriculture(aoi, flooded, after_end, polarization)
urban, urban_affected, urban_area_ha = Urban(LC, aoi, flooded_res)

# Build the Map
This section builds the map, layer-by-layer. 

In [7]:
Map = geemap.Map(center = center, zoom = zoom)
Map.add_basemap('Esri.NatGeoWorldMap')

In [8]:
# Extract VizParams from function
(floodVizParam, populationCountVis, populationExposedVis, 
 LCVis, croplandVis, urbanVis) = vizParamStore()

In [9]:
Map.addLayer(before_filtered, {'min':-25, 'max':0}, 'Before Flood',0);
Map.addLayer(after_filtered, {'min':-25, 'max':0}, 'After Flood',1);
# Difference layer
Map.addLayer(difference,{'min':0,'max':2},"Difference Layer", 0);
# Flooded areas
Map.addLayer(flooded, floodVizParam, 'Flooded areas');
# Population Density
#Map.addLayer(population_count, populationCountVis, 'Population Density', 0);
# Exposed Population
#Map.addLayer(population_exposed, populationExposedVis, 'Exposed Population');
# Landcover Visibility
#Map.addLayer(LC, LCVis, 'Land Cover', 0);
# Cropland
#Map.addLayer(cropland, croplandVis, 'Cropland',0)
# Affected cropland
#Map.addLayer(cropland_affected, croplandVis, 'Affected Cropland'); 
# Urban
#Map.addLayer(urban, urbanVis, 'Urban', 0)
# Affected urban
#Map.addLayer(urban_affected, urbanVis, 'Affected Urban'); 

In [10]:
Map

Map(center=[-19.7234, 34.598], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

In [12]:
roi = aoi.geometry()

#geemap.ee_export_image(
#    difference, filename='difference.tif', scale=10, region=roi, file_per_band=True
#)
